# Load all the librabries

In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import recall_score, accuracy_score, confusion_matrix, precision_recall_fscore_support
import scipy
from sklearn import preprocessing
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix,classification_report, f1_score
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout, Reshape, Activation, BatchNormalization
from keras.optimizers import sgd

from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.linear_model import LassoLarsCV,RidgeClassifierCV
from xgboost.sklearn import XGBRegressor, XGBClassifier
import scipy.io as sio
import lightgbm as lgb
import xgboost as xgb

warnings.filterwarnings('ignore')

seed = 7
np.random.seed(seed)

from subprocess import check_output
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.ensemble import BaggingRegressor, BaggingClassifier
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
import math
import evaluate

Using TensorFlow backend.
C:\Users\SO186018\AppData\Local\Continuum\anaconda3_64\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Load the dataset

In [2]:
dataset = sio.loadmat('./irony.mat')
print(dataset.keys())

dict_keys(['__header__', '__version__', '__globals__', 'skipfeat', 'ngramfeat', 'emojifeat', 'emojisoftfeat', 'hashtagintensityfeat', 'edinberghfeat', 'glovefeat', 'affinfeat', 'binliufeat', 'mpqafeat', 'nrc10efeat', 'nrcemolexfeat', 'nrchashemofeat', 'senti140feat', 'nrchashsentfeat', 'bowfeat', 'tfidffeat', 'structuralfeat', 'sentistrengthfeat', 'trainlabels', 'testlabels', 'trainB', 'testB'])


In [3]:
ironyset = sio.loadmat('./irony_only_test.mat')
print(ironyset.keys())

dict_keys(['__header__', '__version__', '__globals__', 'ngramfeat', 'emojifeat', 'emojisoftfeat', 'edinberghfeat', 'sentistrengthfeat', 'trainlabels'])


In [4]:
print(ironyset['ngramfeat'].shape)

(7201, 300)


In [5]:
largefeatures = ['ngramfeat','emojifeat','emojisoftfeat','edinberghfeat', 'sentistrengthfeat', 'trainlabels']
X_irony = np.concatenate([ironyset['ngramfeat'],ironyset['emojifeat'],ironyset['emojisoftfeat'],ironyset['edinberghfeat'],ironyset['sentistrengthfeat']],axis=1)

Y_irony = ironyset['trainlabels'].T
print(X_irony.shape,Y_irony.shape)

(7201, 3208) (7201, 1)


In [6]:
sarcasmset = sio.loadmat('./sarcasm_only_test.mat')
print(sarcasmset.keys())
print(sarcasmset['sentistrengthfeat'].shape)
X_sarcasm = np.concatenate([sarcasmset['ngramfeat'],sarcasmset['emojifeat'],sarcasmset['emojisoftfeat'],sarcasmset['edinberghfeat'],sarcasmset['sentistrengthfeat']],axis=1)

Y_sarcasm = sarcasmset['trainlabels'].T
print(X_sarcasm.shape,Y_sarcasm.shape)

dict_keys(['__header__', '__version__', '__globals__', 'ngramfeat', 'emojifeat', 'emojisoftfeat', 'edinberghfeat', 'sentistrengthfeat', 'trainlabels'])
(7072, 140)
(7072, 3208) (7072, 1)


In [7]:
humorset = sio.loadmat('./humor_only_test.mat')
print(humorset.keys())
print(humorset['sentistrengthfeat'].shape)
X_humor = np.concatenate([humorset['ngramfeat'],humorset['emojifeat'],humorset['emojisoftfeat'],humorset['edinberghfeat'],humorset['sentistrengthfeat']],axis=1)

Y_humor = humorset['trainlabels'].T
print(X_humor.shape,Y_humor.shape)

dict_keys(['__header__', '__version__', '__globals__', 'ngramfeat', 'emojifeat', 'emojisoftfeat', 'edinberghfeat', 'sentistrengthfeat', 'trainlabels'])
(8491, 140)
(8491, 3208) (8491, 1)


In [64]:
features = ['ngramfeat','emojifeat','emojisoftfeat','hashtagintensityfeat','edinberghfeat', 'glovefeat', 'affinfeat', 'binliufeat', 'mpqafeat', 'nrc10efeat', 'nrcemolexfeat', 'nrchashemofeat', 'senti140feat', 'nrchashsentfeat', 'structuralfeat', 'sentistrengthfeat']
#features1 = ['emojisoftfeat']
X = np.concatenate([dataset[feat] for feat in features],axis=1)
print(X.shape)
Y_train = dataset['trainlabels'].T
Y_trainB = dataset['trainB'].T
print(Y_train.shape, Y_trainB.shape)
no_of_train_sample = Y_train.shape[0]
Y_test = dataset['testlabels'].T
Y_testB = dataset['testB'].T
print(Y_test.shape, Y_testB.shape)
no_of_test_samples = Y_test.shape[0]

X_train = X[0:no_of_train_sample]
print(X_train.shape)
X_test = X[no_of_train_sample:no_of_train_sample+no_of_test_samples]
print(X_test.shape)


from collections import Counter
no_classes = 4
print(no_classes)
from keras.utils import to_categorical
Y_train_cat = to_categorical(Y_trainB, num_classes = no_classes)
print(X_train.shape, Y_train_cat.shape)

(4618, 4636)
(3834, 1) (3834, 1)
(784, 1) (784, 1)
(3834, 4636)
(784, 4636)
4
(3834, 4636) (3834, 4)


In [65]:
X_final = np.concatenate([X_train,X_irony],axis=0)
Y_final = np.concatenate([Y_train,Y_irony],axis=0)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [10]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

class Ensemble1(object):
    def __init__(self, stack, weights='mean', bias=None):
        self.stack = stack
        self.weights = weights
        self.bias = bias
        self.x = None
        
    def fit(self, x_train, y_train, lr_moe=0.001, n_epochs_moe=10):
        len_s = len(self.stack)
        
        for b in self.stack:
            print(type(b))
            #if isinstance(b, KerasClassifier):
            #    y_new = y_train_cat
            #else:
            y_new = y_train
            b.fit(x_train, y_new)
            print('\n')
        
        if self.weights == 'mean':
            self.weights = np.full(len_s, 1./len_s)
        elif self.weights == 'moe':
            print('Train moe algorithm on {}\n'.format(x_train.shape[0]))
            
            self.x = np.random.uniform(low=0, high=1, size=len_s)
            self.bias = np.random.uniform(low=0, high=5, size=len_s)
            
            for t in range(1, n_epochs_moe + 1):                
                y_predict = np.vstack(self.stack_predict(x_train))
                prob = softmax(self.x)        
                
                for k in range(x_train.shape[0]):
                    dG_dx = []
                    dG_db = []
                    
                    for i in range(len_s):                     
                        dG_dx.append(0.5 * prob[i] * (1 - prob[i]) * (y_train - y_predict[i][k] + self.bias[i])**2)
                        dG_db.append(prob[i] * (y_train - y_predict[i][k] + self.bias[i]))
                             
                        self.x[i] -= lr_moe * dG_dx[i][k]
                        self.bias[i] -= lr_moe * dG_db[i][k]
                    
                print('Epoch {}; weights {}; alpha {}'.format(t, softmax(self.x), self.bias))
                
            self.weights = softmax(self.x)
        
    def stack_predict(self, x_valid):
        b = [] 
        for b_ in self.stack:
            b.append(b_.predict(x_valid))
            
        return b
    
    def predict(self, x_valid):              
        predict = np.average(self.stack_predict(x_valid), axis=0, weights=self.weights)
        
        if self.bias != None:
            predict += np.average(self.bias, weights=self.weights, axis=0)
        return predict
  

print('Ensemble class succsessful build')

Ensemble class succsessful build


In [29]:
def create_ml_model():
    GBoost=GradientBoostingClassifier(n_estimators=400,learning_rate=0.1,random_state=100,max_features=4 )
    return GBoost

model_gb = create_ml_model()

xgb_model = xgb.XGBClassifier(max_depth=25, 
                        learning_rate=0.1, 
                        n_estimators=300)

LightGB = lgb.LGBMClassifier(num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
lasso = LassoLarsCV(normalize=True)

rf = RandomForestClassifier(
    n_estimators=100, 
    max_depth=6, 
    min_samples_split=10, 
    class_weight="balanced",
    random_state=100
    )

mlp = MLPClassifier()

ada = AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=4),n_estimators=400, learning_rate=0.1,random_state=100)

clf = svm.SVC(kernel="linear", C=0.35, probability=True, random_state=100)

In [122]:
model = Ensemble1(stack=[mlp,clf1], weights='moe')
model.fit(X_train, Y_train)

<class 'sklearn.neural_network.multilayer_perceptron.MLPClassifier'>


<class 'brew.stacking.stacker.EnsembleStackClassifier'>


Train moe algorithm on 3834

Epoch 1; weights [ 0.5668652  0.4331348]; alpha [ 0.14903159  0.08267374]
Epoch 2; weights [ 0.56651143  0.43348857]; alpha [ 0.01694853  0.01570381]
Epoch 3; weights [ 0.56651215  0.43348785]; alpha [ 0.00193008  0.00297888]
Epoch 4; weights [ 0.56651235  0.43348765]; alpha [ 0.00021979  0.00056507]
Epoch 5; weights [ 0.56651236  0.43348764]; alpha [  2.50298480e-05   1.07189524e-04]
Epoch 6; weights [ 0.56651236  0.43348764]; alpha [  2.85035965e-06   2.03330283e-05]
Epoch 7; weights [ 0.56651236  0.43348764]; alpha [  3.24594465e-07   3.85701909e-06]
Epoch 8; weights [ 0.56651236  0.43348764]; alpha [  3.69643062e-08   7.31646859e-07]
Epoch 9; weights [ 0.56651236  0.43348764]; alpha [  4.20943694e-09   1.38787782e-07]
Epoch 10; weights [ 0.56651236  0.43348764]; alpha [  4.79364045e-10   2.63269750e-08]


In [123]:
Y_pred_model = [int(i) for i in model.predict(X_test)]
p, r, f = evaluate.precision_recall_fscore(Y_true, Y_pred_model, beta=1, labels=[0,1],pos_label=1)
print(p,r,f)
print(classification_report(Y_true,Y_pred_model))
print(confusion_matrix(Y_true,Y_pred_model))
Y_pred_model = model.predict(X_test)
Y_pred_model = [int(i) for i in Y_pred_model]
p, r, f = evaluate.precision_recall_fscore(Y_true, Y_pred_model, beta=1, labels=[0,1],pos_label=1)
print(p,r,f)

print(classification_report(Y_true,Y_pred_model))
print(confusion_matrix(Y_true,Y_pred_model))

0.591304347826087 0.6559485530546624 0.6219512195121951
             precision    recall  f1-score   support

          0       0.76      0.70      0.73       473
          1       0.59      0.66      0.62       311

avg / total       0.69      0.68      0.69       784

[[332 141]
 [107 204]]
0.591304347826087 0.6559485530546624 0.6219512195121951
             precision    recall  f1-score   support

          0       0.76      0.70      0.73       473
          1       0.59      0.66      0.62       311

avg / total       0.69      0.68      0.69       784

[[332 141]
 [107 204]]


In [12]:
import pickle
filename = 'irony_taskA_model_transfer.sav'
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_humor, Y_humor)
#print(result)

In [13]:
Y_true_irony = [int(i) for i in loaded_model.predict(X_irony)]
print(classification_report(Y_irony,Y_true_irony))
print(confusion_matrix(Y_irony,Y_true_irony))

Y_true_sarcasm = [int(i) for i in loaded_model.predict(X_sarcasm)]
print(classification_report(Y_sarcasm,Y_true_sarcasm))
print(confusion_matrix(Y_sarcasm,Y_true_sarcasm))

Y_true_humor = [int(i) for i in loaded_model.predict(X_humor)]
print(classification_report(Y_humor,Y_true_humor))
print(confusion_matrix(Y_humor,Y_true_humor))


             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         0
        1.0       1.00      0.68      0.81      7201

avg / total       1.00      0.68      0.81      7201

[[   0    0]
 [2290 4911]]
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         0
        1.0       1.00      0.68      0.81      7072

avg / total       1.00      0.68      0.81      7072

[[   0    0]
 [2298 4774]]
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         0
        1.0       1.00      0.80      0.89      8491

avg / total       1.00      0.80      0.89      8491

[[   0    0]
 [1660 6831]]


In [20]:
print(np.where(np.array(Y_true_irony)==0)[0])

[   6   10   12 ..., 7180 7186 7189]


In [124]:
Y_true_irony = [int(i) for i in model.predict(X_irony)]
print(classification_report(Y_irony,Y_true_irony))
print(confusion_matrix(Y_irony,Y_true_irony))

Y_true_sarcasm = [int(i) for i in model.predict(X_sarcasm)]
print(classification_report(Y_sarcasm,Y_true_sarcasm))
print(confusion_matrix(Y_sarcasm,Y_true_sarcasm))
#Y_true = [int(i) for i in Y_testB]
#p, r, f = evaluate.precision_recall_fscore(Y_true, Y_pred, beta=1, labels=[0,1,2,3])
#print(p,r,f)

             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         0
        1.0       1.00      0.68      0.81      7201

avg / total       1.00      0.68      0.81      7201

[[   0    0]
 [2290 4911]]
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         0
        1.0       1.00      0.68      0.81      7072

avg / total       1.00      0.68      0.81      7072

[[   0    0]
 [2298 4774]]


In [118]:
print(classification_report(Y_true,Y_pred))
Y_true = [int(i) for i in Y_testB]
p, r, f = evaluate.precision_recall_fscore(Y_true, Y_pred, beta=1, labels=[0,1,2,3])
print(p,r,f)

             precision    recall  f1-score   support

          0       0.61      0.88      0.72       473
          1       0.10      0.04      0.06       164
          2       0.14      0.06      0.08        85
          3       0.00      0.00      0.00        62

avg / total       0.41      0.55      0.46       784

0.21424889543446246 0.24577830539218215 0.21678018162393162


In [179]:
# second feature matrix
X_train2 = pd.DataFrame( {'moe': model.predict(X_train),
     #'GBT': gbr.predict(X_train),
    #'GBTT':GBoost.predict(X_train),
    # 'LGBT': LightGB.predict(X_train),
    #'ET': extreg.predict(X_train),
    })

In [180]:
reg = linear_model.LogisticRegression()
reg1 = linear_model.Ridge()
reg.fit(X_train2, Y_train)

# prediction using the test set
X_test2 = pd.DataFrame( {'moe': model.predict(X_test),
     #'GBT': gbr.predict(X_test),
    #'GBTT': GBoost.predict(X_test),
     #'LGBT': LightGB.predict(X_test),
    #'ET': extreg.predict(X_test),
    })
# Don't forget to convert the prediction back to non-log scale
Y_test_pred = reg.predict(X_test2)
Y_test_reg_new = [int(i) for i in Y_test]
#y_test_pred = [round(float(i),3) for i in Y_test_pred]
print(Y_test_pred[0:10],Y_test_reg_new[0:10])
p, r, f = evaluate.precision_recall_fscore(Y_true, Y_test_pred, beta=1, labels=[0,1], pos_label=1)
print(p,r,f)

[0 1 0 1 0 1 0 1 0 0] [0, 1, 0, 0, 1, 0, 1, 0, 0, 1]
0.5497737556561086 0.7813504823151125 0.6454183266932271


In [80]:
import brew
from brew.base import Ensemble
from brew.combination.combiner import Combiner
from brew.stacking.stacker import EnsembleStack
from brew.stacking.stacker import EnsembleStackClassifier
 
layer_1 = [#svm.SVC(probability=True),
           XGBClassifier(max_depth=6,n_estimators=100),
           RandomForestClassifier(n_estimators=100), 
           MLPClassifier(),
           #GradientBoostingClassifier()
          ]
 
layer_2 = [linear_model.LogisticRegression(max_iter=500), MLPClassifier()]
 
stack1 = EnsembleStack(cv=0) # number of folds per layer
stack1.add_layer(Ensemble(layer_1))
stack1.add_layer(Ensemble(layer_2))
#stack1.add_layer(Ensemble(model))
clf1 = EnsembleStackClassifier(stack1, Combiner('mean'))

In [35]:
clf1.fit(os_features, os_labels)
clf1.predict_proba(X_test)

NameError: name 'os_features' is not defined

In [20]:
print(classification_report(Y_testB, clf.predict(X_test)))

             precision    recall  f1-score   support

          0       0.71      0.79      0.74       473
          1       0.50      0.70      0.58       164
          2       0.69      0.21      0.32        85
          3       0.00      0.00      0.00        62

avg / total       0.61      0.64      0.61       784



In [48]:
clf.fit(X_train, Y_train)
clf.predict_proba(X_test)

array([[ 0.99770174,  0.00229826],
       [ 0.77650473,  0.22349527],
       [ 0.6277022 ,  0.3722978 ],
       ..., 
       [ 0.99340277,  0.00659723],
       [ 0.05285746,  0.94714254],
       [ 0.99003292,  0.00996708]])

In [49]:
print(classification_report(Y_test, clf.predict(X_test)))

             precision    recall  f1-score   support

          0       0.74      0.66      0.70       473
          1       0.55      0.65      0.60       311

avg / total       0.67      0.65      0.66       784



In [101]:
Y_true = [int(i) for i in Y_test]
Y_trueB = [int(i) for i in Y_testB]

In [51]:
p, r, f = evaluate.precision_recall_fscore(Y_true, clf.predict(X_test), beta=1, labels=[0,1], pos_label=1)

In [52]:
print(p,r,f)

0.5537190082644629 0.6463022508038585 0.5964391691394658


In [102]:
sm = SMOTE(ratio='auto',kind='regular')
for i in features3:
    print (i)
    XX = dataset[i][0:no_of_train_sample]
    XT = dataset[i][no_of_train_sample:no_of_train_sample+no_of_test_samples] 
    #os_features,os_labels=sm.fit_sample(XX,Y_train)
    clf1.fit(XX, Y_train)
    p, r, f = evaluate.precision_recall_fscore(Y_true, clf1.predict(XT), beta=1, labels=[0,1], pos_label=1)
    print(p,r,f)

hashtagintensityfeat
0.0 0.0 0.0
structuralfeat
0.47072072072072074 0.6720257234726688 0.5536423841059602


In [121]:
features1 = ['skipfeat','emojifeat','ngramfeat','emojisoftfeat','edinberghfeat', 'glovefeat','bowfeat','tfidffeat', 'binliufeat', 'mpqafeat','nrcemolexfeat', 'nrchashemofeat', 'senti140feat', 'sentistrengthfeat']
features2 = ['emojifeat','ngramfeat','emojisoftfeat','edinberghfeat','sentistrengthfeat']
#features3 = ['hashtagintensityfeat','structuralfeat']

X = np.concatenate([dataset[feat] for feat in features2],axis=1)
print(X.shape)
Y_train = dataset['trainlabels'].T
Y_trainB = dataset['trainB'].T
print(Y_train.shape, Y_trainB.shape)
no_of_train_sample = Y_train.shape[0]
Y_test = dataset['testlabels'].T
Y_testB = dataset['testB'].T
print(Y_test.shape, Y_testB.shape)
no_of_test_samples = Y_test.shape[0]

X_train = X[0:no_of_train_sample]
print(X_train.shape)
X_test = X[no_of_train_sample:no_of_train_sample+no_of_test_samples]
print(X_test.shape)


from collections import Counter
no_classes = 4
print(no_classes)
from keras.utils import to_categorical
Y_train_cat = to_categorical(Y_trainB, num_classes = no_classes)
print(X_train.shape, Y_train_cat.shape)

(4618, 3208)
(3834, 1) (3834, 1)
(784, 1) (784, 1)
(3834, 3208)
(784, 3208)
4
(3834, 3208) (3834, 4)


In [112]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
sm = SMOTE(ratio='auto',kind='regular')
os_features,os_labels=sm.fit_sample(X_train,Y_train)
#us_features,us_labels = NearMiss(ratio='auto').fit_sample(X_train,Y_trainB)

In [146]:
Y_train2 = [int(i) for i in Y_trainB]
Y_train2 = np.array(Y_train2)
clf1.fit(X_train, Y_train)
p, r, f = evaluate.precision_recall_fscore(Y_true, clf1.predict(X_test), beta=1, labels=[0,1],pos_label=1)
print(p,r,f)
print(classification_report(Y_true,clf1.predict(X_test)))
print(confusion_matrix(Y_true,clf1.predict(X_test)))

0.5680628272251309 0.6977491961414791 0.6262626262626264
             precision    recall  f1-score   support

          0       0.77      0.65      0.70       473
          1       0.57      0.70      0.63       311

avg / total       0.69      0.67      0.67       784

[[308 165]
 [ 94 217]]


In [ ]:
p, r, f = evaluate.precision_recall_fscore(Y_trueB, clf1.predict(X_test), beta=1, labels=[0,1,2,3])
print(p,r,f)
print(classification_report(Y_trueB,clf1.predict(X_test)))
print(confusion_matrix(Y_trueB,clf1.predict(X_test)))